In [16]:
import json
import os
import transformers
import matplotlib as plt
import pandas as pd

In [4]:
# Load in the data
node_data = {}
with open('/home/chef/src/DAT6003/data/processed/final_nlp_data.jsonl', 'r') as f:
    for line in f:
        node_data.update(json.loads(line))

node_data

{'0': {'created_at': '2018-11-05T21:35:51Z',
  'author': 'ZhaoyueCheng',
  'author_location': None,
  'type': 'issue',
  'text': 'Thanks a lot for the port! I have some minor questions, for the run_squad file, I see two options for accumulating gradients, accumulate_gradients and gradient_accumulation_steps but it seems to me that it can be combined into one. The other one is for the global_step variable, seems we are only counting but not using this variable in gradient accumulating. Thanks again!'},
 '0_0': {'created_at': '2018-11-05T22:01:47Z',
  'author': 'ZhaoyueCheng',
  'author_location': None,
  'type': 'comment',
  'parent_issue_id': '0',
  'text': 'It also seems to me that the SQuAD VERSION can not reproduce the google tensorflow version performance.'},
 '0_1': {'created_at': '2018-11-05T23:20:46Z',
  'author': 'abeljim',
  'author_location': None,
  'type': 'comment',
  'parent_issue_id': '0',
  'text': 'It also seems to me that the SQuAD VERSION can not reproduce the google

In [5]:
# Transform dict as a DataFrame
df = pd.DataFrame.from_dict(node_data, orient='index')
df.head()

,created_at,author,author_location,type,text,parent_issue_id
0,2018-11-05T21:35:51Z,ZhaoyueCheng,None,issue,Thanks a lot for the port! I have some minor q...,NaN
0_0,2018-11-05T22:01:47Z,ZhaoyueCheng,None,comment,It also seems to me that the SQuAD VERSION can...,0
0_1,2018-11-05T23:20:46Z,abeljim,None,comment,It also seems to me that the SQuAD VERSION can...,0
0_3,2018-11-06T02:05:22Z,abeljim,None,comment,"Just ran on 1 GPU batch size of 10, the result...",0
0_4,2018-11-06T03:48:21Z,ZhaoyueCheng,None,comment,"Sure, Thanks, I'm checking for the reason too,...",0


In [6]:
# Create transformer pipeline for sentiment analysis
sentiment_model = transformers.pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


In [8]:
texts = df['text'].tolist()
output = sentiment_model(texts, batch_size=8, truncation=True, max_length=512, padding=True)

In [9]:
df["sentiment_label"] = [o["label"] for o in output]
df["sentiment_score"] = [o["score"] for o in output]

In [10]:
df.to_csv('/home/chef/src/DAT6003/data/processed/nlp_sentiment_analysis.csv')